In [1]:
import sys
sys.path.append('..')

In [2]:
from src.Network import NetworkEnv
from src.Preprocessing import TransformDataset
from stable_baselines3 import DQN, A2C, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
from datasets import load_dataset

In [3]:
data = load_dataset("Mireu-Lab/NSL-KDD")
df =  TransformDataset(data['train'].to_pandas())  
df.head()

42


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.0,0.5,0.289855,0.9,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,...,0.098039,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0.0
1,0.0,1.0,0.637681,0.9,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,...,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0.0
2,0.0,0.5,0.710145,0.5,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1.0
3,0.0,0.5,0.347826,0.9,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,...,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0.0
4,0.0,0.5,0.347826,0.9,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,...,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [4]:
max_steps=15000

In [5]:
env = NetworkEnv(Dataframe=df, max_steps=max_steps)

In [6]:
policy = "MlpPolicy"
tts = df.shape[0]
verbosity = 0

In [7]:
init , _ = env.reset()
env_dqn = Monitor(env=env,filename='../logs/train_dqn_log')
model_dqn = DQN(policy=policy, env=env_dqn, verbose=verbosity)
model_dqn.learn(total_timesteps=tts, progress_bar=True, reset_num_timesteps=True)
model_dqn.save('../models/model_dqn')

Output()

In [8]:
init, _ = env.reset()
env_custom = Monitor(env=env, filename='../logs/train_custom_log')
policy_kwargs = dict( net_arch=[32,64,128,256,256])
model_dqn_custom = DQN(policy=policy, env=env_custom, policy_kwargs=policy_kwargs, verbose=verbosity)
model_dqn_custom.learn(total_timesteps=tts, progress_bar=True, reset_num_timesteps=True)
model_dqn_custom.save("../models/model_custom")

Output()

In [9]:
init, _ = env.reset()
env_a2c = Monitor(env=env, filename='../logs/train_a2c_log')
model_a2c = A2C(policy=policy, env=env_a2c, verbose=verbosity)
model_a2c.learn(total_timesteps=tts, progress_bar=True)

Output()

In [10]:
model_a2c.save('../models/model_a2c')

In [11]:
init , _ = env.reset()
env_ppo = Monitor(env=env, filename='../logs/train_ppo_log', override_existing=True)
model_ppo = PPO(policy=policy, env=env_ppo, verbose=verbosity)
model_ppo.learn(total_timesteps=tts, progress_bar=True)

Output()

In [12]:
model_ppo.save('../models/model_ppo')

In [13]:
scaled_test = TransformDataset(Dataset=data['test'].to_pandas())
scaled_test.shape

42


(34394, 42)

In [14]:
test_env = NetworkEnv(Dataframe=scaled_test, max_steps=1000)

In [15]:
init, _ = env.reset()
test_env_dqn = Monitor(env=test_env,filename='../logs/test_dqn_log')
evaluate_policy(model=model_dqn, env=test_env_dqn, return_episode_rewards=True)

([682.0, 708.0, 710.0, 705.0, 710.0, 687.0, 683.0, 664.0, 697.0, 719.0],
 [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000])

In [16]:
init, _ = env.reset()
test_env_a2c = Monitor(env=test_env,filename='../logs/test_a2c_log')
evaluate_policy(model=model_a2c, env=test_env_a2c, return_episode_rewards=True)

([749.0, 682.0, 725.0, 714.0, 718.0, 701.0, 718.0, 732.0, 705.0, 707.0],
 [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000])

In [17]:
init, _ = env.reset()
test_env_ppo = Monitor(env=test_env,filename='../logs/test_ppo_log')
evaluate_policy(model=model_ppo, env=test_env_ppo, return_episode_rewards=True)

([679.0, 690.0, 678.0, 687.0, 688.0, 679.0, 706.0, 688.0, 669.0, 718.0],
 [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000])

In [18]:
init, _ = env.reset()
test_env_custom = Monitor(env=test_env,filename='../logs/test_dqn_custom_log')
evaluate_policy(model=model_dqn_custom, env=test_env_custom, return_episode_rewards=True)

([721.0, 708.0, 725.0, 709.0, 705.0, 718.0, 698.0, 720.0, 743.0, 742.0],
 [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000])

#### Evaluation :


In [19]:
loaded_model_dqn = DQN.load('../models/model_dqn')
loaded_model_a2c = A2C.load('../models/model_a2c')
loaded_model_ppo = PPO.load('../models/model_ppo')

In [20]:
def evaluation(model, df_test):
    """
    Evaluate  model on Test set.
    
    Args:
        model: Trained model
        df_test: Test DataFrame
    
    Returns:
        dict: Evaluation metrics
    """
    attempts, correct = 0, 0
    tp, fp, fn, tn = 0, 0, 0, 0
    i=0
    # Create environment
    env = NetworkEnv(Dataframe=df_test, max_steps=df_test.shape[0])
    Y_test = df_test['class'].values
    
    
    # for i in range(5):
    obs = env.reset()
    obs = obs[0]
    done = False
    while not done:
        # Predict action
        action, *_ = model.predict(obs, deterministic=True)
        
        # Step in environment
        obs, rew, terminated, truncated,_ = env.step(action)
        # Update metrics
        done = terminated or truncated
        attempts += 1
        
        # Accuracy and confusion matrix calculations
        if rew > 0:
            correct += 1
        
        # Binary classification metrics
        if Y_test[i] == 0 and rew == 1:
            tp += 1
        elif Y_test[i] == 0 and rew == 0:
            fp += 1
        elif Y_test[i] == 1 and rew == 0:
            fn += 1
        elif Y_test[i] == 1 and rew == 1:
            tn += 1
        i+=1
        # Break if episode is done
        if done:
            break
    
    # Calculate and print metrics
    metrics = {
        'accuracy': (correct / attempts) * 100,
        'false_positive_rate': (fp / (fp + tn)) * 100 if (fp + tn) > 0 else 0,
        'confusion_matrix': {
            'tp': tp, 'fp': fp,
            'fn': fn, 'tn': tn
        },
        'total_attempts': attempts
    }
    
    print('Validation Results:')
    print(f'Accuracy: {metrics["accuracy"]:.2f}%')
    print(f'False Positive Rate: {metrics["false_positive_rate"]:.2f}%')
    print('Confusion Matrix:')
    print(f'{tp} {fp}')
    print(f'{fn} {tn}')
    print(f'Total test values: {attempts}')
    
    return metrics



In [21]:
metrics = evaluation(loaded_model_dqn, scaled_test)

Validation Results:
Accuracy: 68.98%
False Positive Rate: 19.19%
Confusion Matrix:
8169 3694
6976 15555
Total test values: 34394


In [22]:
metrics = evaluation(loaded_model_a2c, scaled_test)

Validation Results:
Accuracy: 70.87%
False Positive Rate: 17.76%
Confusion Matrix:
8417 3446
6574 15957
Total test values: 34394


In [23]:
metrics = evaluation(loaded_model_ppo, scaled_test)

Validation Results:
Accuracy: 69.18%
False Positive Rate: 18.91%
Confusion Matrix:
8235 3628
6971 15560
Total test values: 34394


In [24]:
metrics = evaluation(model_dqn_custom, scaled_test)


Validation Results:
Accuracy: 71.79%
False Positive Rate: 17.02%
Confusion Matrix:
8553 3310
6391 16140
Total test values: 34394
